In [1]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.nan)
from scipy.stats import *

In [2]:
dfa = pd.DataFrame.from_csv(path='/Users/elenahaffmans/Documents/-CogSci3/experiment/data/Group1okt30.csv', sep=',')
dfb = pd.DataFrame.from_csv(path='/Users/elenahaffmans/Documents/-CogSci3/experiment/data/Group2nov3.csv', sep=',')

#add column group:
dfa['Group'] = dfa.index
dfb['Group'] = dfb.index

print(dfa.shape)
print(dfb.shape)

//anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


(984309, 32)
(1008911, 32)


Once we concatenate, the FixationIndex is not unique anymore. 
So first do the StudioEventData naming and the deleting of first fixations.

In [3]:
#for dfa
#create new unique indexing
indexing = list(range(dfa.shape[0]))
dfa = dfa.set_index([indexing])

name='empty'
row=0
for val in dfa.StudioEventData:
    if pd.isnull(val):
        dfa.set_value(dfa.index[row],'StudioEventData',name)
    else:
        name= val
        #print(name)
    row+=1
#print(dfa[:40])


In [4]:
#for dfb
#create new unique indexing
indexing = list(range(dfb.shape[0]))
dfb = dfb.set_index([indexing])

name='empty'
row=0
for val in dfb.StudioEventData:
    if pd.isnull(val):
        dfb.set_value(dfb.index[row],'StudioEventData',name)
    else:
        name= val
        #print(name)
    row+=1
print(dfb[:10])

  ParticipantName StudioEventData  FixationIndex  GazeEventDuration  \
0           PP002           empty            NaN                NaN   
1           PP002           empty            1.0              558.0   
2           PP002           empty            1.0              558.0   
3           PP002           empty            1.0              558.0   
4           PP002           empty            1.0              558.0   
5           PP002           empty            1.0              558.0   
6           PP002           empty            1.0              558.0   
7           PP002           empty            1.0              558.0   
8           PP002           empty            1.0              558.0   
9           PP002           empty            1.0              558.0   

   AOI[normal]Hit  AOI[lit]Hit  AOI[lit]Hit.1  AOI[fig]Hit  AOI[normal]Hit.1  \
0             NaN          NaN            NaN          NaN               NaN   
1             NaN          NaN            NaN          NaN

Delete first fixation for each event

In [5]:
#for every new StudioEventData get the first FixationIndex
groupeda = dfa.groupby(['StudioEventData'], axis=0)['FixationIndex'].min()

#delete all rows where FixationIndex has one of these values.
for val in groupeda:
    dfa= dfa[dfa.FixationIndex != val]

#same for dfb:
groupedb = dfb.groupby(['StudioEventData'], axis=0)['FixationIndex'].min()
for val in groupedb:
    dfb= dfb[dfb.FixationIndex != val]

In [6]:
print(dfa.shape)
print(dfb.shape)
df = pd.concat([dfa,dfb])
print(df.shape)

(968099, 32)
(987100, 32)
(1955199, 32)


We have to make sure that there are the same amount of observations in group A and group B for the ANOVA test.

We first delete all duplicates and events when fixations are not on the AOI

In [7]:
#delete all rows that are duplicates 
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

#first put all values together in 3 collumns (fig, lit and norm)
df = df.fillna(0)

#all data in 3 new column NaN now.
df['AOI_lit'] =    df['AOI[lit]Hit'] + df['AOI[lit]Hit.1'] +df['AOI[lit]Hit.2'] +df['AOI[lit]Hit.3'] +df['AOI[lit]Hit.4'] +df['AOI[lit]Hit.5'] +df['AOI[lit]Hit.6'] +df['AOI[lit]Hit.7'] +df['AOI[lit]Hit.8'] 
df['AOI_fig'] =    df['AOI[fig]Hit'] + df['AOI[fig]Hit.1'] +df['AOI[fig]Hit.2'] +df['AOI[fig]Hit.3'] +df['AOI[fig]Hit.4'] +df['AOI[fig]Hit.5'] +df['AOI[fig]Hit.6'] +df['AOI[fig]Hit.7'] +df['AOI[fig]Hit.8']
df['AOI_normal'] = df['AOI[normal]Hit'] + df['AOI[normal]Hit.1'] +df['AOI[normal]Hit.2'] +df['AOI[normal]Hit.3'] +df['AOI[normal]Hit.4'] +df['AOI[normal]Hit.5'] +df['AOI[normal]Hit.6'] +df['AOI[normal]Hit.7'] +df['AOI[normal]Hit.8']

print('shape after adding 3 columns',df.shape)

#delete all other columns
df = df.drop(['AOI[fig]Hit','AOI[normal]Hit', 'AOI[lit]Hit', 'AOI[lit]Hit.1', 'AOI[normal]Hit.1',
 'AOI[fig]Hit.1' ,'AOI[lit]Hit.2' ,'AOI[lit]Hit.3', 'AOI[normal]Hit.2',
 'AOI[fig]Hit.2', 'AOI[normal]Hit.3', 'AOI[fig]Hit.3' ,'AOI[normal]Hit.4',
 'AOI[fig]Hit.4', 'AOI[lit]Hit.4', 'AOI[normal]Hit.5', 'AOI[normal]Hit.6',
 'AOI[fig]Hit.5', 'AOI[normal]Hit.7', 'AOI[lit]Hit.5', 'AOI[lit]Hit.6',
 'AOI[fig]Hit.6', 'AOI[fig]Hit.7', 'AOI[lit]Hit.7', 'AOI[normal]Hit.8',
 'AOI[fig]Hit.8' ,'AOI[lit]Hit.8'], axis=1)

print(df.shape)

#Drop all rows where df["AOI_lit"], df["AOI_fig"] and df["AOI_normal"] all are NaN.
#create new column with all AOI looks combined, and delete when this column has Nan entry.
df["Any_AOI"]= df['AOI_lit']+ df['AOI_fig']+ df['AOI_normal']
df = df.replace(0,np.nan)
df = df.dropna(axis=0, subset=['Any_AOI'])
df = df.drop(['Any_AOI'], axis=1)

print('amount unique AOI fixations:', df.shape)

(1955199, 32)
(58021, 32)
shape after adding 3 columns (58021, 35)
(58021, 8)
amount unique AOI fixations: (1895, 8)


Now look at what participants are in there, and look at how many Events per group (these have to be the same).
We see that 3 questions are missing, 2 in G1 and 1 in G2. Therefor we delete observations in the other categories of the same idiom. We are left with  123 observations per cat in stead of 126.

In [8]:
df2=df
print(df2['ParticipantName'].unique())

#create column wirh group info
grouped = df2.groupby(['Group','StudioEventData','ParticipantName'], axis=0)['StudioEventData'].describe() 

#For missing value G1 question1 PP003 (figurative)
df2 = df2[~((df2.ParticipantName =='PP002') & (df2.StudioEventData=='all texts no questions.2_1'))]#G2
df2 = df2[~((df2.ParticipantName=='PP006') & (df2.StudioEventData=='all texts no questions.3_1'))]#G1
    
#For missing value G1 question 27 PP014 (normal)
df2 = df2[~((df2.ParticipantName=='PP008') & (df2.StudioEventData=='all texts no questions.25_1'))]#G1
df2 = df2[~((df2.ParticipantName=='PP011') & (df2.StudioEventData=='all texts no questions.26_1'))]#G2

#For missing value G2 question 44 PP004 (literal)
df2 = df2[~((df2.ParticipantName=='p012') & (df2.StudioEventData=='all texts no questions.43_1'))]#G1
df2 = df2[~((df2.ParticipantName=='pp018') & (df2.StudioEventData=='all texts no questions.45_1'))]#G2

#create column wirh group info
grouped = df2.groupby(['Group','StudioEventData','ParticipantName'], axis=0)['StudioEventData'].describe() 

#pretty print ALL results
#with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
#    print(grouped)

df=df2

#pd.DataFrame.to_csv(df,'/Users/elenahaffmans/Documents/-CogSci3/experiment/data/all_processed.csv')
#HEt lijkt erop dat groep 2 nu een obseratie meer heeft.

['PP003' 'PP006' 'PP008' 'PP010' 'p012' 'PP014' 'pp017' 'PP002' 'PP004'
 'PP007' 'pp009' 'PP011' 'PP013' 'pp018']


In [11]:
### FOR TOTAL READING TIME ###

#get df with only lit AOIs
lit = df[df['AOI_lit'] == 1]

#group by story(page) an pp and get total gaze per story, per person
group_lit = lit.groupby(['ParticipantName','StudioEventData'], axis=0)['GazeEventDuration'].sum()
gaze_p_story_lit = group_lit.values

#14 pp who read 9 literal idioms = 126 observations, but turns out exactly one missing everywhere, so 125 of each cat.
print( 'total reading time p literal idiom in ms=', gaze_p_story_lit.sum()/(123)) 
print('literal std=', gaze_p_story_lit.std())

#Same for fig and normal:

fig = df[df['AOI_fig'] == 1]

#per event, maar nog niet per pp:
group_fig = fig.groupby(['ParticipantName','StudioEventData'], axis=0)['GazeEventDuration'].sum() 
gaze_p_story_fig = group_fig.values

#REALLY BAD BUT ADD ONE VALUE BECAUSE ONE OBSERVATION FROM THAT LIST IS MISSING!
#gaze_p_story_fig = np.append(gaze_p_story_fig, 1627)
print('total reading time p figurative idiom in ms=', gaze_p_story_fig.sum()/(123))

print('figurative std=', gaze_p_story_fig.std())

norm = df[df['AOI_normal'] == 1]
group_norm = norm.groupby(['ParticipantName','StudioEventData'], axis=0)['GazeEventDuration'].sum()
gaze_p_story_norm = group_norm.values

print( 'total reading time p normal idiom in ms=', gaze_p_story_norm.sum()/(123))
print('literal norm=', gaze_p_story_norm.std())

'''want to do multivariate repeated Anova measure, but only one way Anova as function
Try it for fixation count and total readig time separate now.'''

#total variance to use for ETA squared.
list_all = list(gaze_p_story_norm) + list(gaze_p_story_norm) + list(gaze_p_story_norm)
variance_all = np.var(list_all)
print('Variance for all data together:', variance_all)

#ANOVA for total readting time:

df_p_story = pd.DataFrame({'lit':gaze_p_story_lit,'fig':gaze_p_story_fig,'normal':gaze_p_story_norm })
#print(df_p_story.head())
f_val, p_val = stats.f_oneway(df_p_story.lit, df_p_story.fig, df_p_story.normal)

print('\n',f_val, p_val)

# Turkey 

from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

#make a list with only data and one with labels.
values = np.append(gaze_p_story_lit, [gaze_p_story_fig, gaze_p_story_norm])

labellit = ['literal' for x in np.array(range(123))]
labelfig = ['figurative' for x in np.array(range(123))]
labelnorm = ['normal' for x in np.array(range(123))]
labels = np.append(labellit, [labelfig, labelnorm])

mc = MultiComparison(values, labels)
result = mc.tukeyhsd()
 
print(result)
print(mc.groupsunique)

total reading time p literal idiom in ms= 1279.92682927
literal std= 851.019705625
total reading time p figurative idiom in ms= 1484.07317073
figurative std= 1233.73619988
total reading time p normal idiom in ms= 1613.85365854
literal norm= 1050.85399791
Variance for all data together: 1104294.12493

 3.0954072273 0.0464434584162
  Multiple Comparison of Means - Tukey HSD,FWER=0.05  
  group1    group2  meandiff   lower    upper   reject
------------------------------------------------------
figurative literal -204.1463  -522.59  114.2973 False 
figurative  normal  129.7805 -188.6631 448.2241 False 
 literal    normal  333.9268  15.4832  652.3705  True 
------------------------------------------------------
['figurative' 'literal' 'normal']


In [12]:
### FOR FIXATION COUNT ### 

#get df with only lit AOIs
lit = df[df['AOI_lit'] == 1]

#now we want to count how many unique fixations there where.
group_lit1 = lit.groupby(['StudioEventData','ParticipantName'], axis=0)['FixationIndex']
tot_fix_lit = group_lit1.count().values
print('lit=', tot_fix_lit.sum()/123)
print('lit std=', tot_fix_lit.std())


group_fig1 = fig.groupby(['StudioEventData','ParticipantName'], axis=0)['FixationIndex']
tot_fix_fig = group_fig1.count().values

#AGAIN ADDING THE AVERAGE TO THE LIST: 6
#tot_fix_fig = np.append(tot_fix_fig, 6)
print('fig=', tot_fix_fig.sum()/123)
print('fig std=', tot_fix_fig.std())
norm1 = df[df['AOI_normal'] == 1] # for some reason otherwise error

group_norm1 = norm.groupby(['StudioEventData','ParticipantName'], axis=0)['FixationIndex']
tot_fix_norm = group_norm1.count().values
print('norm=', tot_fix_norm.sum()/123)
print('norm std=', tot_fix_norm.std())

#variance for eta squared:
all_list = list(tot_fix_norm)+list(tot_fix_lit)+list(tot_fix_fig)
print('Variance for all data together:', np.var(all_list))

## ANOVA
df_fixp_story = pd.DataFrame({'lit':tot_fix_lit,'fig':tot_fix_fig,'normal':tot_fix_norm })
print(df_fixp_story.head())
f_val, p_val = stats.f_oneway(df_fixp_story.lit, df_fixp_story.fig, df_fixp_story.normal)
print(f_val, p_val)


from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

#make a list with only data and one with labels.
values = np.append(tot_fix_lit,[tot_fix_fig, tot_fix_norm])
print(len(tot_fix_lit))
print(len(tot_fix_fig))
print(len(tot_fix_norm))

labellit = ['literal' for x in np.array(range(123))]
labelfig = ['figurative' for x in np.array(range(123))]
labelnorm = ['normal' for x in np.array(range(123))]
labels = np.append(labellit, [labelfig, labelnorm])

mc = MultiComparison(values, labels)
result = mc.tukeyhsd()
 
print(result)
print(mc.groupsunique)

lit= 4.48780487805
lit std= 2.04170283489
fig= 5.0081300813
fig std= 3.40132909211
norm= 5.69105691057
norm std= 3.02867080588
Variance for all data together: 8.54625039475
   fig  lit  normal
0    4    3       4
1    2    2       5
2    3    8       3
3    4    3       4
4    4    2       5
5.35042852973 0.00512459216677
123
123
123
Multiple Comparison of Means - Tukey HSD,FWER=0.05
  group1    group2 meandiff  lower  upper  reject
-------------------------------------------------
figurative literal -0.5203  -1.3886 0.348  False 
figurative  normal  0.6829  -0.1854 1.5512 False 
 literal    normal  1.2033   0.335  2.0715  True 
-------------------------------------------------
['figurative' 'literal' 'normal']


In [13]:
## FOR INITIAL READING TIME ##

group_lit2 = lit.groupby(['ParticipantName','StudioEventData'], axis=0)#['FixationIndex','GazeEventDuration']

'''for each AOI sum the gaze time as long as the fixation index de vorige opvolgt(i.e. niet weggekeken)'''
lit_initial = []
for (fixIndex, gaze), group in group_lit2['FixationIndex', 'GazeEventDuration']:
    idx=0
    for fix in group['FixationIndex']:
        if fix == group['FixationIndex'].min(): #fix is fixation index
            count = group['GazeEventDuration'].iloc[0] 
            idx+=1
            fix_idx = fix
        elif fix-1 == fix_idx:
            count += group['GazeEventDuration'].iloc[idx]
            fix_idx = fix
            idx=idx+1
        else:
            break
    lit_initial.append(count) #now append initial time to a list of all observations

print('literal= ', np.average(np.asarray(lit_initial)))
print('lit std=',(np.asarray(lit_initial)).std())

#same for fig and normal:

group_fig2 = fig.groupby(['ParticipantName','StudioEventData'], axis=0)#['FixationIndex','GazeEventDuration']

'''for each AOI sum the gaze time as long as the fixation index de vorige opvolgt(i.e. niet weggekeken)'''
fig_initial = []
for (fixIndex, gaze), group in group_fig2['FixationIndex', 'GazeEventDuration']:
    idx=0
    for fix in group['FixationIndex']:
        if fix == group['FixationIndex'].min(): #fix is fixation index
            count = group['GazeEventDuration'].iloc[0] 
            idx+=1
            fix_idx = fix
        elif fix-1 == fix_idx:
            count += group['GazeEventDuration'].iloc[idx]
            fix_idx = fix
            idx=idx+1
        else:
            break
    fig_initial.append(count) #now append initial time to a list of all observations

print('figuratve = ', np.average(np.asarray(fig_initial)))
print('fig std=',(np.asarray(fig_initial)).std())


group_norm2 = norm.groupby(['ParticipantName','StudioEventData'], axis=0)#['FixationIndex','GazeEventDuration']

'''for each AOI sum the gaze time as long as the fixation index de vorige opvolgt(i.e. niet weggekeken)'''
norm_initial = []
for (fixIndex, gaze), group in group_norm2['FixationIndex', 'GazeEventDuration']:
    idx=0
    for fix in group['FixationIndex']:
        if fix == group['FixationIndex'].min(): #fix is fixation index
            count = group['GazeEventDuration'].iloc[0] 
            idx+=1
            fix_idx = fix
        elif fix-1 == fix_idx:
            count += group['GazeEventDuration'].iloc[idx]
            fix_idx = fix
            idx=idx+1
        else:
            break
    norm_initial.append(count) #now append initial time to a list of all observations

print('normal =', np.average(np.asarray(norm_initial)))
print('norm std=',(np.asarray(norm_initial)).std())

#Vairance of all for callculation ETA squared done in excel.
variance_all = np.var(np.array(norm_initial + lit_initial + fig_initial))
print('Variance for all data together:',variance_all)

## ANOVA
df_init_story = pd.DataFrame({'lit':lit_initial,'fig':fig_initial,'normal':norm_initial })
print(df_init_story.head())
f_val, p_val = stats.f_oneway(df_init_story.lit, df_init_story.fig, df_init_story.normal)
print(f_val, p_val)

literal=  811.617886179
lit std= 651.759675898
figuratve =  891.634146341
fig std= 716.072329865
normal = 878.18699187
norm std= 696.18878093
Variance for all data together: 475300.233327
      fig     lit  normal
0   316.0  1499.0   525.0
1   400.0   250.0   208.0
2  1041.0   874.0   774.0
3   308.0   325.0  1024.0
4   192.0   691.0   191.0
0.472432540213 0.623863445357




PART II:
POST HOC TESTING - 
Figurative IDIOMS WITH 1 and 2 VERSUS 3 and 4 SCORE

In [14]:
##G1: ['PP003' 'PP006' 'PP008' 'PP010' 'p012' 'PP014' 'pp017']
##G2: ['PP002' 'PP004' 'PP007' 'pp009' 'PP011' 'PP013' 'pp018']

df3 = df

#create column wirh group info
grouped = df3.groupby(['Group','StudioEventData','ParticipantName'], axis=0)['StudioEventData'].describe() 
#pretty print ALL results
#with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
#    print(grouped)
    
#correct figurative sentences rating 1
a = df3[((df3.ParticipantName =='PP011') & (df3.StudioEventData=='all texts no questions.4_1'))]
b = df3[((df3.ParticipantName =='pp009') & (df3.StudioEventData=='all texts no questions.16_1'))]
c = df3[((df3.ParticipantName =='pp017') & (df3.StudioEventData=='all texts no questions.19_1'))]
d = df3[((df3.ParticipantName =='PP013') & (df3.StudioEventData=='all texts no questions.34_1'))]
e = df3[((df3.ParticipantName =='PP004') & (df3.StudioEventData=='all texts no questions.40_1'))]
f = df3[((df3.ParticipantName =='pp009') & (df3.StudioEventData=='all texts no questions.40_1'))]
g = df3[((df3.ParticipantName =='PP011') & (df3.StudioEventData=='all texts no questions.46_1'))]

#rating 2
aa = df3[((df3.ParticipantName =='pp009') & (df3.StudioEventData=='all texts no questions.10_1'))]
bb = df3[((df3.ParticipantName =='PP002') & (df3.StudioEventData=='all texts no questions.16_1'))]
cc = df3[((df3.ParticipantName =='PP002') & (df3.StudioEventData=='all texts no questions.22_1'))]
dd = df3[((df3.ParticipantName =='PP014') & (df3.StudioEventData=='all texts no questions.31_1'))]
ee = df3[((df3.ParticipantName =='pp009') & (df3.StudioEventData=='all texts no questions.34_1'))]
ff = df3[((df3.ParticipantName =='PP011') & (df3.StudioEventData=='all texts no questions.34_1'))]
gg = df3[((df3.ParticipantName =='pp018') & (df3.StudioEventData=='all texts no questions.40_1'))]
hh = df3[((df3.ParticipantName =='pp018') & (df3.StudioEventData=='all texts no questions.52_1'))]
#concatenate these df's
df_1 = pd.concat([a,b,c,d,e,f,g,aa,bb,cc,dd,ee,ff,gg,hh])

#matching 1's
i = df3[((df3.ParticipantName =='pp018') & (df3.StudioEventData=='all texts no questions.4_1'))]
j = df3[((df3.ParticipantName =='PP013') & (df3.StudioEventData=='all texts no questions.16_1'))]
k = df3[((df3.ParticipantName =='PP014') & (df3.StudioEventData=='all texts no questions.19_1'))]
l = df3[((df3.ParticipantName =='PP004') & (df3.StudioEventData=='all texts no questions.34_1'))]
m = df3[((df3.ParticipantName =='PP002') & (df3.StudioEventData=='all texts no questions.40_1'))]
n = df3[((df3.ParticipantName =='pp018') & (df3.StudioEventData=='all texts no questions.46_1'))]
o = df3[((df3.ParticipantName =='PP007') & (df3.StudioEventData=='all texts no questions.40_1'))]

#matching rating 2
ii = df3[((df3.ParticipantName =='PP004') & (df3.StudioEventData=='all texts no questions.10_1'))]
jj = df3[((df3.ParticipantName =='PP007') & (df3.StudioEventData=='all texts no questions.16_1'))]
kk = df3[((df3.ParticipantName =='pp009') & (df3.StudioEventData=='all texts no questions.22_1'))]
ll = df3[((df3.ParticipantName =='PP006') & (df3.StudioEventData=='all texts no questions.31_1'))]
mm = df3[((df3.ParticipantName =='PP007') & (df3.StudioEventData=='all texts no questions.34_1'))]
nn = df3[((df3.ParticipantName =='pp018') & (df3.StudioEventData=='all texts no questions.34_1'))]
oo = df3[((df3.ParticipantName =='PP013') & (df3.StudioEventData=='all texts no questions.40_1'))]
pp = df3[((df3.ParticipantName =='PP013') & (df3.StudioEventData=='all texts no questions.52_1'))]

df_4 = pd.concat([i,j,k,l,m,n,o,ii,jj,kk,ll,mm,nn,oo,pp])
print(mm)

        FixationIndex  GazeEventDuration    Group ParticipantName  \
472312         3406.0              258.0  group 2           PP007   
472343         3407.0              425.0  group 2           PP007   

                    StudioEventData  AOI_lit  AOI_fig  AOI_normal  
472312  all texts no questions.34_1      NaN      1.0         NaN  
472343  all texts no questions.34_1      NaN      1.0         NaN  


In [15]:
##TOtal reading time
from scipy import stats

#group by story(page) an pp and get total gaze per story, per person
group_1 = df_1.groupby(['ParticipantName','StudioEventData'], axis=0)['GazeEventDuration'].sum()
gaze_p_story_1 = group_1.values
print(gaze_p_story_1)

group_4 = df_4.groupby(['ParticipantName','StudioEventData'], axis=0)['GazeEventDuration'].sum()
gaze_p_story_4 = group_4.values
print(gaze_p_story_4)

ttest = scipy.stats.ttest_ind(gaze_p_story_1,gaze_p_story_4)
print(ttest)
##Total fixation count

group_11 = df_1.groupby(['StudioEventData','ParticipantName'], axis=0)['FixationIndex']
tot_fix_1 = group_11.count().values
print('G1=', tot_fix_1.sum()/5)
group_44 = df_4.groupby(['StudioEventData','ParticipantName'], axis=0)['FixationIndex']
tot_fix_4 = group_44.count().values
print('G4=', tot_fix_4.sum()/5)

print(scipy.stats.ttest_ind(tot_fix_1,tot_fix_4))

##For initial reading time

group_111 = df_1.groupby(['ParticipantName','StudioEventData'], axis=0)#['FixationIndex','GazeEventDuration']

'''for each AOI sum the gaze time as long as the fixation index de vorige opvolgt(i.e. niet weggekeken)'''
G1_initial = []
for (fixIndex, gaze), group in group_111['FixationIndex', 'GazeEventDuration']:
    idx=0
    for fix in group['FixationIndex']:
        if fix == group['FixationIndex'].min(): #fix is fixation index
            count = group['GazeEventDuration'].iloc[0] 
            idx+=1
            fix_idx = fix
        elif fix-1 == fix_idx:
            count += group['GazeEventDuration'].iloc[idx]
            fix_idx = fix
            idx=idx+1
        else:
            break
    G1_initial.append(count) #now append initial time to a list of all observations

print(G1_initial)

group_444 = df_4.groupby(['ParticipantName','StudioEventData'], axis=0)#['FixationIndex','GazeEventDuration']

'''for each AOI sum the gaze time as long as the fixation index de vorige opvolgt(i.e. niet weggekeken)'''
G4_initial = []
for (fixIndex, gaze), group in group_444['FixationIndex', 'GazeEventDuration']:
    idx=0
    for fix in group['FixationIndex']:
        if fix == group['FixationIndex'].min(): #fix is fixation index
            count = group['GazeEventDuration'].iloc[0] 
            idx+=1
            fix_idx = fix
        elif fix-1 == fix_idx:
            count += group['GazeEventDuration'].iloc[idx]
            fix_idx = fix
            idx=idx+1
        else:
            break
    G4_initial.append(count) #now append initial time to a list of all observations

print(G4_initial)


print(scipy.stats.ttest_ind(tot_fix_1,tot_fix_4))


[ 1932.  1831.  5111.  3781.  1782.  1408.  4973.   433.   466.   658.
  1273.  3630.  1506.  1215.  1132.]
[ 6461.  1108.  3224.   433.   782.   683.  1957.  1924.  2647.  6262.
   774.   658.  2973.  2523.   566.]


NameError: name 'scipy' is not defined